In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools, offline
plotly.tools.set_credentials_file(username='mtodisco10', api_key='bCfUmq5FralbymKXY4uX')
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
%run progress_bar.ipynb

In [4]:
#Read in Data
game_data = pd.read_csv('game_data.csv')
play_info = pd.read_csv('play_information.csv')
play_player_role = pd.read_csv('play_player_role_data.csv')
player_punt = pd.read_csv('player_punt_data.csv')
video_footage_control = pd.read_csv('video_footage-control.csv')
video_footage_injury = pd.read_csv('video_footage-injury.csv')
video_review = pd.read_csv('video_review.csv')

In [45]:
NGS_2016_pre = pd.read_csv('NGS-2016-pre.csv')
NGS_2016_post = pd.read_csv('NGS-2016-post.csv')
NGS_2017_pre = pd.read_csv('NGS-2017-pre.csv')
NGS_2017_post = pd.read_csv('NGS-2017-post.csv')

In [6]:
NGS_2016_wk1_6 = pd.read_csv('NGS-2016-reg-wk1-6.csv')
NGS_2016_wk7_12 = pd.read_csv('NGS-2016-reg-wk7-12.csv')
NGS_2016_wk13_17 = pd.read_csv('NGS-2016-reg-wk13-17.csv')

In [48]:
NGS_2017_wk1_6 = pd.read_csv('NGS-2017-reg-wk1-6.csv')
NGS_2017_wk7_12 = pd.read_csv('NGS-2017-reg-wk7-12.csv')
NGS_2017_wk13_17 = pd.read_csv('NGS-2017-reg-wk13-17.csv')

In [8]:
NGS_2016 = pd.concat([NGS_2016_pre, NGS_2016_post, \
                      NGS_2016_wk1_6, NGS_2016_wk7_12, \
                      NGS_2016_wk13_17])
del NGS_2016_pre, NGS_2016_post, NGS_2016_wk13_17, NGS_2016_wk7_12, NGS_2016_wk1_6

In [49]:
NGS_2017 = pd.concat([NGS_2017_pre, NGS_2017_post, \
                      NGS_2017_wk1_6, NGS_2017_wk7_12, \
                      NGS_2017_wk13_17])
del NGS_2017_pre, NGS_2017_post, NGS_2017_wk13_17, NGS_2017_wk7_12, NGS_2017_wk1_6

In [52]:
NGS_df = pd.concat([NGS_2016, NGS_2017])

### Injury Plays EDA

In [11]:
'There are concussion injuries on ' + str(round(len(video_review) / float(len(play_info)) * 100, 2)) + '% of ' + 'punt plays'

'There are concussion injuries on 0.55% of punt plays'

##### It is good for the NFL and players that injuries only occur on less than 1% of punt plays, but it will be hard to do analysis

#### How are players being injured?

In [12]:
trace1 = go.Bar(
        x=video_review.groupby(['Player_Activity_Derived'], as_index=False)['PlayID'].count()['Player_Activity_Derived'],
        y=video_review.groupby(['Player_Activity_Derived'], as_index=False)['PlayID'].count()['PlayID']
    )
trace2 = go.Bar(
        x=video_review.groupby(['Primary_Impact_Type'], as_index=False)['PlayID'].count()['Primary_Impact_Type'],
        y=video_review.groupby(['Primary_Impact_Type'], as_index=False)['PlayID'].count()['PlayID'],
    )
trace3 = go.Bar(
        x=video_review.groupby(['Friendly_Fire'], as_index=False)['PlayID'].count()['Friendly_Fire'],
        y=video_review.groupby(['Friendly_Fire'], as_index=False)['PlayID'].count()['PlayID'],
    )
trace4 = go.Bar(
        x=video_review.groupby(['Primary_Partner_Activity_Derived'], as_index=False)['PlayID'].count()['Primary_Partner_Activity_Derived'],
        y=video_review.groupby(['Primary_Partner_Activity_Derived'], as_index=False)['PlayID'].count()['PlayID'],
    )

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Player Activity Derived', 'Primary Impact Type',
                                                          'Friendly Fire', 'Primary Partner Activity Derived'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(showlegend=False)

py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



### Quick Conclusions
    - No player activity stands out
    - Helmet to player is primary impact type
    - Friendly fire is only clear 16% of the time
    - No Partner Activity Stands Out

#### Who's getting hurt?

In [13]:
#Categorize player punt roles into the sides of the ball
return_roles = ['PDL1','PDL2','PDL3','PDL4','PDL5','PDL6','PDM','PDR1','PDR2','PDR3','PDR4','PDR5','PDR6'
                ,'PFB','PLL','PLL1','PLL2','PLL3','PLM','PLM1','PLR','PLR1','PLR2','PLR3','PR','VL','VLi'
                ,'VLo','VR','VRi','VRo']

coverage_roles = ['GL','GLi','GLo','GR','GRi','GRo','P','PC','PLG','PLS','PLT','PLW','PPL','PPLi','PPLo'
                 ,'PPR','PPRi','PPRo','PRG','PRT','PRW']

gunner_blockers = ['VL','VLi','VLo','VR','VRi','VRo']

In [14]:
inj_players = video_review.merge(play_player_role, how='inner', on=['Season_Year', 'GameKey', 'PlayID', 'GSISID'])
inj_players.rename(columns={'Role':'inj_role'}, inplace=True)

In [15]:
inj_players['inj_side_of_ball'] = np.where(inj_players.inj_role.isin(return_roles), 'return',
                                          np.where(inj_players.inj_role.isin(coverage_roles), 'coverage', ''))

In [16]:
partner_players = video_review[['Season_Year', 'GameKey', 'PlayID', 'Primary_Partner_GSISID']]
partner_players['Primary_Partner_GSISID'] = partner_players.loc[:,'Primary_Partner_GSISID'] \
                                                .replace('Unclear','0').fillna(0).astype(int)
partner_players = partner_players.merge(play_player_role, how='left', \
                                        left_on=['Season_Year', 'GameKey', 'PlayID', 'Primary_Partner_GSISID'],\
                                       right_on =['Season_Year', 'GameKey', 'PlayID', 'GSISID'])
partner_players = partner_players.drop('GSISID', axis=1)
partner_players.rename(columns={'Role':'partner_role'}, inplace=True)
partner_players['partner_side_of_ball'] = np.where(partner_players.partner_role.isin(return_roles), 'return',
                                          np.where(partner_players.partner_role.isin(coverage_roles), 'coverage', ''))
inj_partner_df = pd.concat([inj_players,partner_players[['partner_role','partner_side_of_ball']]], axis = 1)

In [17]:
inj_partner_df.head()

,Season_Year,GameKey,PlayID,GSISID,Player_Activity_Derived,Turnover_Related,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived,Friendly_Fire,inj_role,inj_side_of_ball,partner_role,partner_side_of_ball
0,2016,5,3129,31057,Tackling,No,Helmet-to-body,32482,Tackled,No,PLW,coverage,PR,return
1,2016,21,2587,29343,Blocked,No,Helmet-to-helmet,31059,Blocking,No,GL,coverage,PLL1,return
2,2016,29,538,31023,Tackling,No,Helmet-to-body,31941,Tackled,No,GR,coverage,PR,return
3,2016,45,1212,33121,Tackling,No,Helmet-to-body,28249,Tackled,No,PRT,coverage,PR,return
4,2016,54,1045,32444,Blocked,No,Helmet-to-body,31756,Blocked,Yes,PRT,coverage,GR,coverage


In [18]:
partner_players.partner_side_of_ball.value_counts()

return      18
coverage    15
             4
Name: partner_side_of_ball, dtype: int64

#### On what types of plays are players getting hurt?

In [19]:
#Create a column in the play_info dataframe to determine the type of play
play_info['outcome'] = np.where(play_info['PlayDescription'].str.contains('fair catch', flags=re.IGNORECASE, regex=True), 'fair_catch',
                        np.where(play_info['PlayDescription'].str.contains('touchback', flags=re.IGNORECASE, regex=True), 'touchback',
                        np.where(play_info['PlayDescription'].str.contains('out of bounds', flags=re.IGNORECASE, regex=True), 'oob',
                        np.where(play_info['PlayDescription'].str.contains('downed', flags=re.IGNORECASE, regex=True), 'downed',
                        np.where(play_info['PlayDescription'].str.contains('blocked', flags=re.IGNORECASE, regex=True), 'blocked',
                        np.where(play_info['PlayDescription'].str.contains('fake|pass', flags=re.IGNORECASE, regex=True), 'fake', 
                        np.where(play_info['PlayDescription'].str.contains('safety', flags=re.IGNORECASE, regex=True), 'safety',
                        np.where(play_info['PlayDescription'].str.contains('[0-9]+ for [-+]?[0-9]+ yards?|for no gain|muffs|touchdown', flags=re.IGNORECASE, regex=True), 'return',         
                        np.where(play_info['PlayDescription'].str.contains('- no play|delay of game|false start, declined|penalty enforced', flags=re.IGNORECASE, regex=True), 'no_play', ' ')))))))))

In [20]:
pi = play_info.merge(video_review[['Season_Year', 'GameKey', 'PlayID','GSISID']], how='left', on =['Season_Year', 'GameKey', 'PlayID'])
pi['injury'] = np.where(pi.GSISID.notnull(), 1, 0).astype(int)
pi.drop('GSISID', axis = 1, inplace=True)
vr = play_info.merge(video_review, how='inner', on =['Season_Year', 'GameKey', 'PlayID'])

In [21]:
vr.outcome.value_counts()

return        32
downed         3
fair_catch     2
Name: outcome, dtype: int64

#### Conclusion: players are getting hurt on returns 

In [22]:
#Is it a higher proportion compared to all plays?

In [23]:
pi_inj_grouped = pi.groupby(['outcome'], as_index=False)['injury'] \
    .agg({'total_plays':'count','injuries':sum}) \
    .sort_values('total_plays', ascending = False) \
    .reset_index(drop=True)

pi_inj_grouped['injury_percentage'] = round(pi_inj_grouped['injuries'] / pi_inj_grouped['total_plays'] * 100, 1).astype(str) + '%'
pi_inj_grouped

,outcome,total_plays,injuries,injury_percentage
0,return,2917,32,1.1%
1,fair_catch,1663,2,0.1%
2,downed,802,3,0.4%
3,oob,696,0,0.0%
4,touchback,407,0,0.0%
5,no_play,139,0,0.0%
6,blocked,26,0,0.0%
7,fake,25,0,0.0%
8,safety,6,0,0.0%


#### Conclusion: over 1% of punt plays with a return have an injury
    - **** This is 10x higher than when a fair catch is called

#### Plot Player Paths on a Given Play

In [29]:
def load_layout():
    """
    Returns a dict for a Football themed Plot.ly layout 
    """
    layout = dict(
        title = "Player Activity",
        plot_bgcolor='darkseagreen',
        showlegend=True,
        xaxis=dict(
            autorange=False,
            range=[0, 120],
            showgrid=False,
            zeroline=False,
            showline=True,
            linecolor='black',
            linewidth=1,
            mirror=True,
            ticks='',
            tickmode='array',
            tickvals=[10,20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
            ticktext=['Goal', 10, 20, 30, 40, 50, 40, 30, 20, 10, 'Goal'],
            showticklabels=True
        ),
        yaxis=dict(
            title='',
            autorange=False,
            range=[-3.3,56.3],
            showgrid=False,
            zeroline=False,
            showline=True,
            linecolor='black',
            linewidth=1,
            mirror=True,
            ticks='',
            showticklabels=False
        ),
        shapes=[
            dict(
                type='line',
                layer='below',
                x0=0,
                y0=0,
                x1=120,
                y1=0,
                line=dict(
                    color='white',
                    width=2
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=0,
                y0=53.3,
                x1=120,
                y1=53.3,
                line=dict(
                    color='white',
                    width=2
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=10,
                y0=0,
                x1=10,
                y1=53.3,
                line=dict(
                    color='white',
                    width=10
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=20,
                y0=0,
                x1=20,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=30,
                y0=0,
                x1=30,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=40,
                y0=0,
                x1=40,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=50,
                y0=0,
                x1=50,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=60,
                y0=0,
                x1=60,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),dict(
                type='line',
                layer='below',
                x0=70,
                y0=0,
                x1=70,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),dict(
                type='line',
                layer='below',
                x0=80,
                y0=0,
                x1=80,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=90,
                y0=0,
                x1=90,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),dict(
                type='line',
                layer='below',
                x0=100,
                y0=0,
                x1=100,
                y1=53.3,
                line=dict(
                    color='white'
                )
            ),
            dict(
                type='line',
                layer='below',
                x0=110,
                y0=0,
                x1=110,
                y1=53.3,
                line=dict(
                    color='white',
                    width=10
                )
            )
        ]
    )
    return layout

layout = load_layout()

In [33]:
# Loading and plotting functions

def load_plays_for_game(GameKey):
    """
    Returns a dataframe of play data for a given game (GameKey)
    """
    play_information = pd.read_csv('play_information.csv')
    play_information = play_information[play_information['GameKey'] == GameKey]
    return play_information


def load_game_and_ngs(ngs_file=None, GameKey=None):
    """
    Returns a dataframe of player movements (NGS data) for a given game
    """
    if ngs_file is None:
        print("Specifiy an NGS file.")
        return None
    if GameKey is None:
        print('Specify a GameKey')
        return None
    # Merge play data with NGS data    
    plays = load_plays_for_game(GameKey)
    ngs = pd.read_csv(ngs_file, low_memory=False)
    merged = pd.merge(ngs, plays, how="inner", on=["GameKey", "PlayID", "Season_Year"])
    return merged


def plot_play(game_df, PlayID, player1=None, player2=None, custom_layout=False):
    """
    Plots player movements on the field for a given game, play, and two players
    """
    game_df = game_df[game_df.PlayID==PlayID]
    
    GameKey=str(pd.unique(game_df.GameKey)[0])
    HomeTeam = pd.unique(game_df.Home_Team_Visit_Team)[0].split("-")[0]
    VisitingTeam = pd.unique(game_df.Home_Team_Visit_Team)[0].split("-")[1]
    YardLine = game_df[(game_df.PlayID==PlayID) & (game_df.GSISID==player1)]['YardLine'].iloc[0]
    
    traces=[]   
    if (player1 is not None) & (player2 is not None):
        game_df = game_df[ (game_df['GSISID']==player1) | (game_df['GSISID']==player2)]
        for player in pd.unique(game_df.GSISID):
            player = int(player)
            trace = go.Scatter(
                x = game_df[game_df.GSISID==player].x,
                y = game_df[game_df.GSISID==player].y,
                name='GSISID '+str(player),
                mode='markers'
            )
            traces.append(trace)
    else:
        print("Specify GSISIDs for player1 and player2")
        return None
    
    if custom_layout is not True:
        layout = load_layout()
        layout['title'] =  HomeTeam + \
        ' vs. ' + VisitingTeam + \
        '<br>Possession: ' + \
        YardLine.split(" ")[0] +'@'+YardLine.split(" ")[1]
    data = traces
    fig = dict(data=data, layout=layout)
    play_description = game_df[(game_df.PlayID==PlayID) & (game_df.GSISID==player1)].iloc[0]["PlayDescription"]
    print("\n\n\t",play_description)
    #py.iplot(fig,filename='jupyter-table1')
    offline.iplot(fig)

In [522]:
game280 = load_game_and_ngs('NGS-2016-reg-wk13-17.csv',GameKey=280)

In [523]:
plot_play(game_df=game280, PlayID=2918, player1=32120, player2=32725)



	 (3:36) B.Nortman punts 49 yards to HST 30, Center-C.Tinker, fair catch by W.Fuller.


#### Calculate hang time of punts

In [36]:
def get_hang_time(ngs_df, start_event='punt', *stop_events):
    punt_event = ngs_df.loc[ngs_df.Event==start_event] \
        .groupby(['Season_Year', 'GameKey','PlayID'], as_index = False)['Time'].min()
    punt_event.rename(columns = {'Time':'punt_time'}, inplace=True)
    punt_event['punt_time'] = pd.to_datetime(punt_event['punt_time'],\
                                             format='%Y-%m-%d %H:%M:%S.%f')
    
    receiving_event = ngs_df.loc[ngs_df.Event.isin(stop_events)] \
        .groupby(['Season_Year', 'GameKey','PlayID'], as_index = False)['Time'].min()
    receiving_event.rename(columns = {'Time':'receiving_time'}, inplace=True)
    receiving_event['receiving_time'] = pd.to_datetime(receiving_event['receiving_time'],\
                                             format='%Y-%m-%d %H:%M:%S.%f')
    
    punt_df = punt_event.merge(receiving_event, how='inner', on = ['Season_Year','GameKey','PlayID']) \
                .reset_index(drop=True)
    
    punt_df['hang_time'] = (punt_df['receiving_time'] - punt_df['punt_time']).dt.total_seconds()
    
    return punt_df

In [37]:
punt_df = get_hang_time(NGS_2016, 'punt', 'punt_received', 'fair_catch')

In [38]:
data = [go.Histogram(x=punt_df.hang_time)]

py.iplot(data, filename='basic histogram')

In [39]:
round(punt_df['hang_time'].mean(), 1)

4.5

In [40]:
round(punt_df['hang_time'].median(), 1)

4.5

In [41]:
str(round(len(punt_df.loc[punt_df.hang_time < 5.5]) / len(punt_df) * 100, 1)) + '% of hang times are less than 5 1/2 seconds'

'97.1% of hang times are less than 5 1/2 seconds'

### Calculate distance between returner and closest coverage man

In [43]:
#Create a function to calculate the space between the returner and the closest coverage player for every second between the punt and the catch
def coverage_returner_space(play_df, ngs_df):
    cov_ret_lst = []
    play_df = play_df.loc[(play_df.outcome == 'fair_catch') | (play_df.outcome == 'return')].reset_index(drop=True)
    for i in log_progress(range(0, len(play_df)), every=25):
        season_key = play_df['Season_Year'][i]
        game_key = play_df['GameKey'][i]
        play_id = play_df['PlayID'][i]
        outcome = play_df['outcome'][i]
        injury = play_df['injury'][i]
        if game_key in ngs_df.GameKey and play_id in ngs_df.PlayID:
            filtered_play = ngs_df.loc[(ngs_df.GameKey == game_key) \
                                       & (ngs_df.PlayID == play_id)].sort_values('Time').reset_index(drop=True)
            filtered_play = filtered_play.merge(play_player_role, \
                                                how='inner', on = ['Season_Year','GameKey','PlayID','GSISID'])
            if len(filtered_play) > 0:
                filtered_play['Time'] = pd.to_datetime(filtered_play['Time'], \
                                                       format='%Y-%m-%d %H:%M:%S.%f')
                punt_event_time = filtered_play.loc[filtered_play.Event == 'punt'].Time.min()
                receiving_event_time = filtered_play.loc[(filtered_play.Event == 'punt_received') | \
                                                         (filtered_play.Event == 'fair_catch')].Time.min()
                gunner_blocker_count = len(filtered_play.loc[filtered_play['Role'].isin(gunner_blockers)]['Role'].unique())
                filtered_play = filtered_play.loc[(filtered_play.Time >= punt_event_time) & \
                                                  (filtered_play.Time <= receiving_event_time)]
                coverage_df = filtered_play.loc[filtered_play['Role'].isin(coverage_roles)].sort_values('Time')
                coverage_df.rename(columns={'x':'cov_x', 
                                          'y': 'cov_y',
                                          'GSISID': 'cov_GSISID',
                                          'dis': 'cov_dis',
                                          'o': 'cov_o',
                                          'dir': 'cov_dir',
                                          'Role': 'cov_Role'
                                         }, inplace=True)
                
                returner_df = filtered_play.loc[filtered_play['Role'] == 'PR'].sort_values('Time')
                returner_df.rename(columns={'x':'ret_x', 
                                          'y': 'ret_y',
                                          'GSISID': 'ret_GSISID',
                                          'dis': 'ret_dis',
                                          'o': 'ret_o',
                                          'dir': 'ret_dir',
                                          'Role': 'ret_Role'
                                         }, inplace=True)
                returner_df = returner_df.drop('Event', axis = 1)
                
                cov_ret_df = coverage_df.merge(returner_df, how ='inner', on = ['Season_Year','GameKey','PlayID','Time'])
                cov_ret_df['dis_from_ret'] = ((cov_ret_df['cov_x'] -  cov_ret_df['ret_x']) ** 2 \
                                           + (cov_ret_df['cov_y'] -  cov_ret_df['ret_y']) ** 2).apply(np.sqrt)
                cov_ret_df['time_since_punt'] = cov_ret_df['Time'] - punt_event_time
                times_to_capture = [punt_event_time + pd.Timedelta(seconds=i) for i in range(1, 7)]
                cov_ret_df = cov_ret_df.loc[cov_ret_df['Time'].isin(times_to_capture)]
                cov_ret_df['gunner_blockers'] = gunner_blocker_count
                cov_ret_df['outcome'] = outcome
                cov_ret_df['injury'] = injury
                cov_ret_df['cov_speed'] = convert_to_mph(cov_ret_df.cov_dis, 20.455)
                cov_ret_df['ret_speed'] = convert_to_mph(cov_ret_df.ret_dis, 20.455)
                if len(cov_ret_df) > 0:
                    cov_ret_lst.append(cov_ret_df)
                    
    cov_ret_df = pd.concat(cov_ret_lst).reset_index(drop=True)          
    return cov_ret_df

In [44]:
def convert_to_mph(dis_vector, converter):
    mph_vector = dis_vector * converter
    return mph_vector

In [54]:
cov_ret_df = coverage_returner_space(pi, NGS_df)

In [55]:
cov_ret_df.gunner_blockers.value_counts(normalize=True)

2    0.525857
3    0.335905
4    0.121392
0    0.008650
1    0.007823
5    0.000373
Name: gunner_blockers, dtype: float64

In [56]:
#Speed vs Number of Gunner Blockers at each second.

pd.pivot_table(cov_ret_df.loc[(cov_ret_df.cov_Role == 'GR') | (cov_ret_df.cov_Role == 'GL')], 
               index=['gunner_blockers'], values='cov_speed',
               columns=['time_since_punt'], aggfunc=[np.mean, np.median])

mean                                              \
time_since_punt   00:00:01   00:00:02   00:00:03   00:00:04   00:00:05   
gunner_blockers                                                          
0                12.453004  14.203952  14.956696  14.354074  11.337914   
1                15.459889  16.008083  15.034425  13.758423   8.256382   
2                17.598287  18.436175  17.535374  15.110873  12.457405   
3                16.073269  17.272085  17.180509  16.026920  13.900728   
4                14.764080  15.960092  15.885425  15.535428  14.074988   
5                 5.011475   7.977450   7.261525   1.636400   6.852425   

                               median                                 \
time_since_punt   00:00:06   00:00:01  00:00:02   00:00:03  00:00:04   
gunner_blockers                                                        
0                      NaN  13.295750  15.95490  17.079925  15.75035   
1                11.966175  17.795850  18.20495  16.977650  15.54580   
2                11.174889  18.000400  19.02315  18.204950  16.15945   
3                11.279471  16.773100  17.79585  17.795850  16.77310   
4                13.398025  15.545800  16.77310  16.773100  16.56855   
5                      NaN   5.011475   7.97745   7.261525   1.63640   

                                       
time_since_punt   00:00:05   00:00:06  
gunner_blockers                        
0                10.943425        NaN  
1                 5.931950  11.966175  
2                13.500301  11.454800  
3                14.523050  10.227500  
4                15.341250  13.398025  
5                 6.852425        NaN

In [57]:
# Average distance for all player at the X second mark.  Return vs Fair Catch
pd.pivot_table(cov_ret_df, values='dis_from_ret', index=['time_since_punt'], columns=['outcome'], aggfunc=[np.mean, np.median])

mean                median           
outcome         fair_catch     return fair_catch     return
time_since_punt                                            
00:00:01         45.093671  48.330382  45.788646  48.504611
00:00:02         39.879917  43.495235  40.572556  43.454322
00:00:03         32.809098  37.429303  33.047457  37.093847
00:00:04         25.907994  31.867596  25.304660  31.138620
00:00:05         22.514878  26.713022  20.971050  25.289009
00:00:06               NaN  22.261890        NaN  20.015158

In [58]:
pd.pivot_table(cov_ret_df, values='dis_from_ret', index=['time_since_punt'], columns=['outcome','gunner_blockers'], aggfunc=np.mean)

outcome         fair_catch                                              \
gunner_blockers          0          1          2          3          4   
time_since_punt                                                          
00:00:01         47.566563  46.480602  44.433250  46.875262  46.160511   
00:00:02         44.363982  41.602827  39.320604  41.417205  40.308591   
00:00:03         38.519668  34.287975  32.481350  33.808522  32.230327   
00:00:04         31.803258  27.173324  25.861231  26.228923  24.508603   
00:00:05         27.710252  21.354172  23.194379  21.465353  21.528805   
00:00:06               NaN        NaN        NaN        NaN        NaN   

outcome             return                                              \
gunner_blockers          0          1          2          3          4   
time_since_punt                                                          
00:00:01         47.387136  49.335898  48.284114  48.474168  48.082217   
00:00:02         43.947354  45.224487  43.514626  43.557361  43.174846   
00:00:03         39.091520  39.142888  37.536414  37.419758  37.007094   
00:00:04         33.486391  32.857348  32.170371  31.732481  31.329589   
00:00:05         23.076005  25.378162  26.854060  26.464621  27.381261   
00:00:06               NaN  12.567207  22.374571  22.591091  23.919940   

outcome                     
gunner_blockers          5  
time_since_punt             
00:00:01         49.855188  
00:00:02         44.907907  
00:00:03         38.183061  
00:00:04         32.561592  
00:00:05         28.111912  
00:00:06               NaN

In [59]:
#### At each second of each play... Who is the closest to the returner?  How far away are they?  Return vs Fair Catch

In [60]:
min_dist_from_ret = cov_ret_df.groupby(['Season_Year', 'GameKey','PlayID','time_since_punt'], as_index = False)['dis_from_ret'].min()

In [61]:
min_dist_from_ret.head()

,Season_Year,GameKey,PlayID,time_since_punt,dis_from_ret
0,2016,66,1607,00:00:01,38.975477
1,2016,66,1607,00:00:02,31.414700
2,2016,66,1607,00:00:03,21.148130
3,2016,66,2241,00:00:01,44.283168
4,2016,66,2241,00:00:02,40.007933


In [62]:
closest_df = cov_ret_df.merge(min_dist_from_ret.drop('time_since_punt', axis = 1), how = 'inner', on=['Season_Year', 'GameKey', 'PlayID', 'dis_from_ret'])

In [63]:
#closest_df.groupby(['time_since_punt','cov_Role'], as_index=False)['dis_from_ret'].agg({'# of times closest': 'count','avg distance':np.mean}).sort_values('# of times closest', ascending =False)

In [64]:
plt.plot(df_grouped.loc[df_grouped['outcome']=='fair_catch']['time_since_punt'], df_grouped.loc[df_grouped['outcome']=='fair_catch']['dis_from_ret'])
plt.plot(df_grouped.loc[df_grouped['outcome']=='return']['time_since_punt'], df_grouped.loc[df_grouped['outcome']=='return']['dis_from_ret'])
plt.legend()

NameError: name 'df_grouped' is not defined

In [65]:
pos_grouped_df = cov_ret_df.groupby(['cov_Role','time_since_punt', 'outcome'], as_index=False)['dis_from_ret'].mean()

In [66]:
trace1 = go.Bar(
        x=pos_grouped_df.loc[pos_grouped_df.outcome == 'fair_catch']['cov_Role'],
        y=pos_grouped_df.loc[pos_grouped_df.outcome == 'fair_catch']['dis_from_ret'],
        name='fair catch'
    )

trace2 = go.Bar(
        x=pos_grouped_df.loc[pos_grouped_df.outcome == 'return']['cov_Role'],
        y=pos_grouped_df.loc[pos_grouped_df.outcome == 'return']['dis_from_ret'],
        name='return'
    )

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    xaxis=dict(title='Position'),
    yaxis= dict(title='Avg Distance From Returner (Yds)')
)

fig=go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

#### Punts Landing inside the 20

In [597]:
inside_twenty = play_info.loc[play_info.outcome.isin(['touchback', 'fair_catch',\
                                                      'oob', 'downed', 'return'])].reset_index(drop=True)

In [602]:
punt_to_lst = []
punt_result_lst = []

In [603]:
for i in range(0, len(inside_twenty)):
    punt_search = re.search('punts [0-9]+ yards? to [A-Z]+ [0-9]+',\
                            inside_twenty.PlayDescription[i])
    return_search = re.search('to [A-Z]+ [0-9]+ for [-+]?[0-9]+ yards?',\
                              inside_twenty.PlayDescription[i])
    
    if inside_twenty.outcome[i] == 'touchback':
        punt_to_lst.append(0)
        punt_result_lst.append(20)

    # if there is a return
    elif inside_twenty.outcome[i] == 'return':
        if 'no gain' in inside_twenty.PlayDescription[i]:
            # punt to and punt result are the same yard line
            nums = re.findall(r'\d+', punt_desc)
            punt_to_lst.append(int(nums[-1]))
            punt_result_lst.append(int(nums[-1]))
        else:
            # there is a return
            if return_search:
                snip = return_search.group(0)
                yd_line = re.findall(r'\d+', snip)[0]
                #yard line after return
                punt_result_lst.append(int(yd_line))
            else:
                punt_result_lst.append('Something is messed up')
            
            if punt_search:
                nums = re.findall(r'\d+', punt_search.group(0))
                punt_to_lst.append(int(nums[-1]))
            else:
                punt_to_lst.append('Something is messed up')
    else:
        if punt_search:   
            nums = re.findall(r'\d+', punt_search.group(0))
            punt_to_lst.append(int(nums[-1]))
            punt_result_lst.append(int(nums[-1]))
        else:
            punt_to_lst.append('MESSY')
            punt_result_lst.append('MESSY')

In [604]:
inside_twenty['punt_to'] = punt_to_lst
inside_twenty['punt_result'] = punt_result_lst

In [607]:
inside_twenty.head()

,Season_Year,Season_Type,GameKey,Game_Date,Week,PlayID,Game_Clock,YardLine,Quarter,Play_Type,Poss_Team,Home_Team_Visit_Team,Score_Home_Visiting,PlayDescription,outcome,punt_to,punt_result
0,2016,Pre,2,08/13/2016,2,191,12:30,LA 47,1,Punt,LA,LA-DAL,0 - 7,"(12:30) J.Hekker punts 52 yards to DAL 1, Cent...",touchback,0,20
1,2016,Pre,2,08/13/2016,2,1132,12:08,LA 29,2,Punt,LA,LA-DAL,7 - 21,"(12:08) J.Hekker punts 51 yards to DAL 20, Cen...",fair_catch,20,20
2,2016,Pre,2,08/13/2016,2,1227,10:01,DAL 18,2,Punt,DAL,LA-DAL,7 - 21,"(10:01) C.Jones punts 40 yards to LA 42, Cente...",return,42,33
3,2016,Pre,2,08/13/2016,2,1864,00:21,DAL 46,2,Punt,LA,LA-DAL,7 - 24,"(:21) J.Hekker punts 31 yards to DAL 15, Cente...",oob,15,15
4,2016,Pre,2,08/13/2016,2,2247,10:26,DAL 15,3,Punt,DAL,LA-DAL,14 - 24,"(10:26) M.Wile punts 40 yards to LA 45, Center...",oob,45,45


In [606]:
for i in inside_twenty.PlayDescription:
    print(i)

(12:30) J.Hekker punts 52 yards to DAL 1, Center-J.McQuaide, downed by LA-E.Sims. Dallas challenged the kick downed ruling, and the play was REVERSED. J.Hekker punts 53 yards to end zone, Center-J.McQuaide, Touchback.
(12:08) J.Hekker punts 51 yards to DAL 20, Center-J.McQuaide, fair catch by L.Whitehead. PENALTY on DAL-K.Wilber, Offensive Holding, 10 yards, enforced at DAL 20.
(10:01) C.Jones punts 40 yards to LA 42, Center-L.Ladouceur. P.Cooper to DAL 33 for 25 yards (C.Jones; A.Hitchens).
(:21) J.Hekker punts 31 yards to DAL 15, Center-J.McQuaide, out of bounds.
(10:26) M.Wile punts 40 yards to LA 45, Center-Z.Wood, out of bounds.
(9:00) J.Hekker punts 50 yards to end zone, Center-J.Overbaugh, Touchback.
(:27) J.Hekker punts 48 yards to DAL 14, Center-J.Overbaugh, fair catch by L.Whitehead. PENALTY on DAL-D.Olatoye, Defensive 12 On-field, 5 yards, enforced at LA 38 - No Play.
(13:30) M.Wile punts 41 yards to end zone, Center-Z.Wood, Touchback.
(5:28) M.Wile punts 59 yards to LA 26, 

(1:06) C.Schmidt punts 37 yards to NE 25, Center-G.Sanborn. C.Jones MUFFS catch, and recovers at NE 26.
(3:11) J.Ryan punts 31 yards to NYJ 10, Center-N.Frese, fair catch by J.Ross.
(:13) L.Edwards punts 33 yards to SEA 8, Center-T.Purdum, fair catch by T.Lockett.
(7:49) L.Edwards punts 37 yards to SEA 15, Center-T.Purdum, out of bounds.
(10:59) J.Ryan punts 56 yards to NYJ 8, Center-N.Frese, fair catch by J.Ross.
(7:52) L.Edwards punts 41 yards to SEA 44, Center-T.Purdum. T.Lockett to SEA 45 for 1 yard (M.Williams; J.Burris).
(2:37) L.Edwards punts 39 yards to SEA 18, Center-T.Purdum, downed by NYJ-C.Pryor. PENALTY on SEA-S.Terrell, Offensive Holding, 9 yards, enforced at SEA 18.
(1:04) J.Ryan punts 48 yards to NYJ 36, Center-N.Frese, fair catch by J.Ross.
(11:22) L.Edwards punts 52 yards to SEA 18, Center-T.Purdum. T.Lockett to SEA 25 for 7 yards (J.Martin; B.Carter).
(5:56) J.Ryan punts 34 yards to NYJ 9, Center-N.Frese, fair catch by J.Ross.
(1:51) J.Ryan punts 34 yards to NYJ 13, 

(13:55) B.Pinion punts 42 yards to ARZ 34, Center-K.Nelson. Jo.Brown ran ob at ARZ 36 for 2 yards (D.Johnson).
(5:34) R.Quigley punts 43 yards to SF 18, Center-A.Brewer. J.Kerley pushed ob at SF 21 for 3 yards (B.Golden).
(4:02) B.Pinion punts 51 yards to ARZ 29, Center-K.Nelson. B.Golden to ARZ 35 for 6 yards (A.Burbridge).
(:52) B.Pinion punts 33 yards to ARZ 13, Center-K.Nelson, fair catch by P.Peterson.
(12:23) R.Quigley punts 41 yards to SF 26, Center-A.Brewer, fair catch by J.Kerley.
(8:19) B.Pinion punts 43 yards to ARZ 4, Center-K.Nelson, downed by SF-M.Cromartie.
(14:38) (Punt formation) J.Berry punts 44 yards to DAL 35, Center-G.Warren. L.Whitehead to DAL 40 for 5 yards (T.Matakevich, A.Moats).
(3:17) (Punt formation) C.Jones punts 49 yards to PIT 18, Center-L.Ladouceur, out of bounds.
(:51) (Punt formation) J.Berry punts 27 yards to DAL 16, Center-G.Warren, fair catch by C.Beasley.
(:27) (Punt formation) C.Jones punts 48 yards to PIT 36, Center-L.Ladouceur, fair catch by A.B

(7:28) K.Redfern punts 52 yards to NYJ 28, Center-D.Muhlbach. M.Murphy to NYJ 28 for no gain (Q.Diggs; J.Bademosi).
(5:29) L.Edwards punts 50 yards to DET 15, Center-T.Purdum. T.Jones to DET 22 for 7 yards (S.Thomas).
(10:41) L.Edwards punts 41 yards to DET 7, Center-T.Purdum. J.Agnew to DET 13 for 6 yards (F.Bishop).
(8:51) K.Redfern punts 51 yards to NYJ 40, Center-D.Muhlbach. M.Murphy to 50 for 10 yards (N.Bellore).
(6:55) L.Edwards punts 36 yards to DET 14, Center-T.Purdum, fair catch by J.Agnew. PENALTY on DET-P.Worrilow, Offensive Holding, 7 yards, enforced at DET 14.
(13:38) K.Redfern punts 45 yards to NYJ 26, Center-D.Muhlbach. F.Hammond MUFFS catch, RECOVERED by DET-R.Tonyan at NYJ 25. R.Tonyan to NYJ 25 for no gain (D.Roberts).
(6:59) K.Redfern punts 35 yards to NYJ 11, Center-D.Muhlbach, fair catch by F.Hammond.
(3:44) L.Edwards punts 50 yards to DET 6, Center-T.Purdum. K.Martin to DET 28 for 22 yards (J.Howsare). PENALTY on NYJ-J.Jenkins, Illegal Formation, 5 yards, enforce

(12:01) J.Berry punts 53 yards to DET 19, Center-K.Canaday. J.Agnew to DET 29 for 10 yards (T.Matakevich).
(5:03) J.Berry punts 44 yards to DET 15, Center-K.Canaday, fair catch by J.Agnew.
(14:16) R.Dixon punts 43 yards to KC 28, Center-C.Kreiter. T.Hill ran ob at DEN 36 for 36 yards. PENALTY on KC-E.Murray, Offensive Holding, 10 yards, enforced at KC 28.
(10:58) D.Colquitt punts 31 yards to DEN 27, Center-J.Winchester. I.McKenzie to DEN 27 for no gain (K.Pierre-Louis).
(8:00) R.Dixon punts 34 yards to KC 32, Center-C.Kreiter, downed by DEN-C.Kreiter.
(10:12) D.Colquitt punts 50 yards to DEN 19, Center-J.Winchester. I.McKenzie to DEN 19 for no gain (J.Chesson).
(9:17) R.Dixon punts 37 yards to KC 46, Center-C.Kreiter, downed by DEN-J.Carter.
(:14) R.Dixon punts 48 yards to KC 10, Center-C.Kreiter, downed by DEN-A.Janovich.
(13:38) D.Colquitt punts 47 yards to DEN 17, Center-J.Winchester. I.McKenzie MUFFS catch, RECOVERED by KC-D.Thomas at DEN 16. D.Thomas to DEN 16 for no gain (I.McKen

In [521]:
for i in inside_twenty.loc[inside_twenty.punt_result == 'Something is messed up'].PlayDescription:
    print(i)

(3:09) J.Locke punts 61 yards to CIN 20, Center-K.McDermott. A.Erickson for 80 yards, TOUCHDOWN.
(8:11) R.Allen punts 50 yards to NO 35, Center-J.Cardona. M.Murphy MUFFS catch, and recovers at NO 37. M.Murphy to NE 15 for 48 yards (G.Grissom).
(12:53) D.Jones punts 38 yards to TB 28, Center-J.DePalma. B.Reedy MUFFS catch, and recovers at TB 35. B.Reedy to TB 38 for 3 yards (J.DePalma).
(11:51) J.Locke punts 56 yards to SEA 23, Center-K.McDermott. M.Hunter MUFFS catch, and recovers at SEA 16. M.Hunter to SEA 11 for -5 yards (T.Plummer; K.Carter).
(:28) P.Mortell punts 54 yards to OAK 15, Center-R.Lovato. J.Mickens MUFFS catch, and recovers at OAK 13. J.Mickens to OAK 11 for -2 yards (B.Martinez). PENALTY on OAK-T.Jones, Tripping, 6 yards, enforced at OAK 11.
(8:41) B.Pinion punts 42 yards to DEN 21, Center-K.Nelson. B.Addison MUFFS catch, and recovers at DEN 21. B.Addison to DEN 24 for 3 yards (K.Nelson).
(1:58) T.Way punts 56 yards to BUF 29, Center-N.Sundberg. R.Bush MUFFS catch, and 

In [516]:
inside_twenty.iloc[114,:]['PlayDescription']

'(3:09) J.Locke punts 61 yards to CIN 20, Center-K.McDermott. A.Erickson for 80 yards, TOUCHDOWN.'

In [515]:
punt_result_lst.index('Something is messed up')

114

In [500]:
len(inside_twenty)

6485

In [485]:
for i in inside_twenty.iloc[13:15,:].PlayDescription:
    print(i)

(6:10) (Punt formation) S.Waters punts 36 yards to BLT 15, Center-J.Jansen. K.Clay MUFFS catch, RECOVERED by CAR-F.Whittaker at BLT 12. F.Whittaker to BLT 12 for no gain (K.Clay).
(1:59) (Punt formation) S.Koch punts 44 yards to CAR 47, Center-M.Cox. D.Byrd MUFFS catch, RECOVERED by BLT-P.Onwuasor at CAR 48. P.Onwuasor to CAR 48 for no gain (T.Robinson). Penalty on CAR-R.McClain, Defensive Holding, declined.


In [296]:
inside_twenty['yard_line'] = punt_yd_line_lst

In [299]:
inside_twenty = inside_twenty.loc[inside_twenty.yard_line <= 20]

In [301]:
inside_twenty.outcome.value_counts(normalize=True)

return        0.393252
fair_catch    0.318712
downed        0.163190
touchback     0.124847
Name: outcome, dtype: float64

In [304]:
for i in play_info.loc[play_info.outcome == 'return'].PlayDescription:
    print(i)

(10:01) C.Jones punts 40 yards to LA 42, Center-L.Ladouceur. P.Cooper to DAL 33 for 25 yards (C.Jones; A.Hitchens).
(6:44) (Punt formation) S.Koch punts 54 yards to CAR 14, Center-M.Cox. D.Byrd to CAR 23 for 9 yards (B.Beyer).
(2:54) (Punt formation) S.Koch punts 45 yards to CAR 21, Center-M.Cox. D.Byrd to CAR 20 for -1 yards (T.Young). PENALTY on CAR-Z.Sanchez, Illegal Block Above the Waist, 10 yards, enforced at CAR 20.
(6:10) (Punt formation) S.Waters punts 36 yards to BLT 15, Center-J.Jansen. K.Clay MUFFS catch, RECOVERED by CAR-F.Whittaker at BLT 12. F.Whittaker to BLT 12 for no gain (K.Clay).
(1:59) (Punt formation) S.Koch punts 44 yards to CAR 47, Center-M.Cox. D.Byrd MUFFS catch, RECOVERED by BLT-P.Onwuasor at CAR 48. P.Onwuasor to CAR 48 for no gain (T.Robinson). Penalty on CAR-R.McClain, Defensive Holding, declined.
(1:53) A.Lee punts 40 yards to GB 27, Center-C.Hughlett. J.Abbrederis to GB 28 for 1 yard (B.Mingo).
(2:48) A.Lee punts 66 yards to GB 15, Center-C.Hughlett. J.Ab

(6:55) (Punt formation) K.Allen punts 57 yards to WAS 26, Center-M.Cox. K.Fuller to WAS 43 for 17 yards (R.Daniel).
(1:25) (Punt formation) K.Allen punts 47 yards to WAS 13, Center-M.Cox. K.Fuller to WAS 16 for 3 yards (B.Bradley).
(:25) (Punt formation) T.Way punts 59 yards to BLT 21, Center-N.Sundberg. K.Reynolds to WAS 33 for 46 yards (T.Way).
(8:20) C.Schmidt punts 57 yards to MIN 11, Center-R.Ferguson. R.Adams to MIN 22 for 11 yards (R.Ferguson; J.Banyard).
(6:01) R.Quigley punts 46 yards to BUF 28, Center-K.McDermott. B.Tate to BUF 34 for 6 yards (B.Gedeon).
(14:08) C.Schmidt punts 46 yards to MIN 23, Center-R.Ferguson. S.Coley to MIN 31 for 8 yards (C.Bradford).
(8:49) C.Schmidt punts 42 yards to MIN 33, Center-R.Ferguson. R.Adams MUFFS catch, recovered by MIN-J.Tocho at MIN 29. J.Tocho to MIN 29 for no gain (T.Jones).
(5:42) R.Quigley punts 45 yards to BUF 44, Center-K.McDermott. R.Ross to BUF 44 for no gain (E.Wilson). PENALTY on BUF-M.Sayles, Illegal Block Above the Waist, 10

(14:08) (Punt formation) B.Kern punts 42 yards to CIN 31, Center-B.Brinkley. A.Jones pushed ob at TEN 46 for 23 yards (W.Woodyard).
(11:17) (Punt formation) B.Kern punts 57 yards to CIN 15, Center-B.Brinkley. A.Jones to CIN 26 for 11 yards (D.Bates).
(6:19) (Punt formation) B.Kern punts 50 yards to CIN 15, Center-B.Brinkley. A.Erickson pushed ob at CIN 30 for 15 yards (E.Weems). PENALTY on CIN-B.Wilson, Offensive Holding, 10 yards, enforced at CIN 20.
(4:45) (Punt formation) K.Huber punts 52 yards to TEN 26, Center-C.Harris. A.Jackson to TEN 35 for 9 yards (J.Evans).
(11:57) (Punt formation) K.Huber punts 43 yards to TEN 24, Center-C.Harris. A.Jackson to TEN 29 for 5 yards (C.Harris).
(7:03) (Punt formation) K.Huber punts 50 yards to TEN 23, Center-C.Harris. A.Jackson to TEN 26 for 3 yards (C.Core).
(5:59) (Punt formation) B.Kern punts 66 yards to CIN 11, Center-B.Brinkley. A.Erickson to CIN 18 for 7 yards (W.Woodyard).
(6:14) T.Way punts 52 yards to MIN 9, Center-N.Sundberg. M.Sherels